<B>This NoteBook is for Segmenting and Clustering Neighborhoods in Toronto.</B></n></n>

Resolve dependancy if any, Install beautifulsoup4 and request pakage. 



In [5]:
!pip install beautifulsoup4
!pip install requests

distributed 1.21.8 requires msgpack, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.


<H3>Steps</H3>

i)   Import required libraries like pd, beautifulSoup, request.

ii)  Get the html request object for wikipedia web of postal codes of canada.

iii) Using beautifulsoup object get table tag data.

iv)  Iterate through HTML table, read all table row <tr>  and table data <td> and create list
    
V)   Create Pandas Dataframe from list.

In [6]:
import pandas as pd           # library to process data as dataframes
from bs4 import BeautifulSoup # library to process html page
import requests               # library to request data object

html = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(html, "lxml")

table = soup.find('table', attrs={'class':'wikitable sortable'}) # Get HTML table tag data
table_rows = table.find_all('tr')                  ## find all table row in table

res = []
for tr in table_rows:                        ## Iterate and create all list from table data.
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

### Create DataFrame from HTML table list in above step and name columns.
df = pd.DataFrame(res, columns=["PostalCode", "Borough", "Neighbourhood"])
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


<H3>Count the size of dataframe after creating from HTML table</H3>

In [3]:
df.shape

(289, 3)

<H3>Steps</H3>

i) Filter rows if any Borough have 'Not assigned' value.

ii) Replace Neighbourhood values with Borough if Neighbourhood is having value 'Not assigned'.

In [7]:
df = df.loc[df['Borough'] != 'Not assigned'].reset_index()  # Filter 'Not assigned' Borough's

df.Neighbourhood.replace('Not assigned',df.Borough,inplace=True)

df.head(10)

,index,PostalCode,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M5A,Downtown Toronto,Regent Park
4,6,M6A,North York,Lawrence Heights
5,7,M6A,North York,Lawrence Manor
6,8,M7A,Queen's Park,Queen's Park
7,10,M9A,Etobicoke,Islington Avenue
8,11,M1B,Scarborough,Rouge
9,12,M1B,Scarborough,Malvern


<H3>Count the size of dataframe after filtering 'Not assigned' Borough</H3>

In [5]:
df.shape

(212, 4)

<H3>Steps</H3>

i) Where more than one neighborhood can exist in one postal code combine two rows into one row with the neighborhoods separated with a comma.

In [8]:
df = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(list).to_frame().reset_index()

df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]
5,M1J,Scarborough,[Scarborough Village]
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]"
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]"
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]"
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]"


<H3>Show the shape of the final data frame</H3>

In [9]:
df.shape

(103, 3)

<H3>Install dependancy geocoder</H3>

In [8]:
!pip install geocoder

    100% |████████████████████████████████| 102kB 18.9MB/s 
distributed 1.21.8 requires msgpack, which is not installed.


<H3>Get longitude and Latitude for PostalCodes of Canada </H3>

In [ ]:

import geocoder # import geocoder

## Itereate
for i, row in df.iterrows():
    
    #initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, {}'.format(row.PostalCode,row.Borough))
        lat_lng_coords = g.latlng
        print(lat_lng_coords)
        #latitude = lat_lng_coords[0]
        #longitude = lat_lng_coords[1]
        #print(latitude,longitude)

<H3>As Geocoder library is not responding hence using CSV file to get latitude and longitude</H3>

In [10]:
dfll = pd.read_csv('Geospatial_Coordinates.csv')
dfll.head(10)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [10]:
dfll.shape

(103, 3)

<h3> Merge the two dataframe on Postal code, show sample data and count</h3>

In [11]:
df = df.merge(dfll,on='PostalCode')


In [28]:
df.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476
5,M1J,Scarborough,[Scarborough Village],43.744734,-79.239476
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]",43.727929,-79.262029
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]",43.711112,-79.284577
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]",43.716316,-79.239476
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]",43.692657,-79.264848


In [13]:
df.shape

(103, 5)

Import dependancy 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

##!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

##!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<H3>Get Toronto cordinates, latitude and longitude</H3>

In [2]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of New York City are 43.653963, -79.387207.


<H3> Get map of toronto, Ontario and superimpose borough </H3>

In [16]:
# create map of Toronto,  using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<H4>The above map generated visualize your neighborhoods of toronto, maked with blue. Neighbourhoods in same Borough are clusted together. </n>
    

Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.</H4>

let's simplify the above map and segment and cluster only the neighborhoods in <B>Central Toronto.</B> So let's slice the original dataframe and create a new dataframe of the <B>Central Toronto.</B>

In [19]:
ct_data = df[df['Borough'] == 'Central Toronto'].reset_index(drop=True)
ct_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,[Lawrence Park],43.728020,-79.388790
1,M4P,Central Toronto,[Davisville North],43.712751,-79.390197
2,M4R,Central Toronto,[North Toronto West],43.715383,-79.405678
3,M4S,Central Toronto,[Davisville],43.704324,-79.388790
4,M4T,Central Toronto,"[Moore Park, Summerhill East]",43.689574,-79.383160


<H4>Let's get the geographical coordinates of <B>Central Toronto</B>.</H4>

In [23]:
address = 'Central Toronto, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Central Toronto are 43.638093, -79.4665843.


<H3>As we did with all of Toronto, let's visualize Central Toronto the neighborhoods in it.</H3>

In [26]:
# create map of Central Toronto using latitude and longitude values
map_ct = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(ct_data['Latitude'], ct_data['Longitude'], ct_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ct)  
    
map_ct

<H3>The above map show Central Toranto borough and neighbourhood in it</H3>